In [1]:
import spacy
import pandas as pd
import xml.etree.ElementTree as ET
import json

# Parse Xml

In [2]:
def parse_xml_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        xml_string = file.read()
    
    tree = ET.ElementTree(ET.fromstring(xml_string))
    root = tree.getroot()

    reviews = []
    sentence_dict = {}
    sentence_id = 0
    for review_elem in root.findall('.//Review'):
        sentence_dict = {}
        sentence_id = 0
        for sentence_elem in review_elem.findall('.//sentence'):
            sentence_data = []
            # sentence_id = 0
            text = sentence_elem.find('text').text
            # print("text=",text)
            # clauses = split_into_clauses(nlp(text))
            

            opinion_elems = sentence_elem.find('Opinions')
            if opinion_elems is not None:
                for opinion_elem in opinion_elems.findall('Opinion'):
                    target = opinion_elem.get('target')
                    # if term == 'NULL':
                    #     continue
                    # for clause in clauses:
                    #     if term in clause:
                    #         break
                    # print(term)
                    # print(clause)
                    # print(clauses)
                    category = opinion_elem.get('category')
                    polarity = opinion_elem.get('polarity')
                    # sentences_data.append((text, category.split('#')[0], polarity))
                    # sentences_data.append((text, category, polarity, target))
                    sentence_data.append((category, polarity, target))
                # print("text=",text)
                sentence_dict[sentence_id] = (text, sentence_data)
                # print(sentence_dict)
                sentence_id += 1
        # print(sentence_dict)
        reviews.append(sentence_dict)




    return reviews

## Train data

In [3]:
# File path
file_path = 'ABSA16_Restaurants_Train_SB1_v2.xml'

# Parse XML file and extract data
train_data = parse_xml_file(file_path)


# pretty_json = json.dumps(reviews, indent=4)
# print(pretty_json)

## Trail data

In [4]:
# File path
file_path = 'restaurants_trial_english_sl.xml'

# Parse XML file and extract data
trail_data = parse_xml_file(file_path)

## Test data

In [5]:
# File path
file_path = 'EN_REST_SB1_TEST_gold.xml'

# Parse XML file and extract data
test_data = parse_xml_file(file_path)

In [6]:
print(len(train_data), len(test_data), len(trail_data))

350 90 10


In [ ]:
def compute_category(data):
    category_sentiment_counts = {}

    for clause, category, sentiment in data:
        if category not in category_sentiment_counts:
            category_sentiment_counts[category] = {'positive': 0, 'neutral': 0, 'negative': 0}

        category_sentiment_counts[category][sentiment] += 1

    return category_sentiment_counts

In [ ]:
category_sentiment_counts = compute_category(sentences_data)
print(category_sentiment_counts)
print(len(category_sentiment_counts))

In [7]:
train_data

[{0: ('Judging from previous posts this used to be a good place, but not any longer.',
   [('RESTAURANT#GENERAL', 'negative', 'place')]),
  1: ('We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.',
   [('SERVICE#GENERAL', 'negative', 'staff')]),
  2: ('They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.',
   [('SERVICE#GENERAL', 'negative', 'NULL')]),
  3: ('The food was lousy - too sweet or too salty and the portions tiny.',
   [('FOOD#QUALITY', 'negative', 'food'),
    ('FOOD#STYLE_OPTIONS', 'negative', 'portions')]),
  4: ('After all that, they complained to me about the small tip.',
   [('SERVICE#GENERAL', 'negative', 'NULL')]),
  5: ('Avoid this place!', [('RESTAURANT#GENERAL', 'negative', 'place')])},
 {0: ('I have eaten at Saul, many times, the food is always consistently, outrageously good.',
   [('FOOD#QUALITY', 'positive',

In [8]:
import pickle

In [9]:
# reviesw_data = (train_data, test_data, trail_data)
# with open("reviews_data", "wb") as fp:   #Pickling
#     pickle.dump(reviesw_data, fp)

# Prepare data for split into clasues using gpt api

In [23]:
for review in test_data:
    for key, sentence_data in review.items():
        if 5 == len(sentence_data[1]):
            print(sentence_data)
            break

("– I don't understand how I was a stranger to this place for so long... the fajita salad, the colorado, the fajitas - EVERYTHING is delicious.", [('FOOD#QUALITY', 'positive', 'fajita salad'), ('FOOD#QUALITY', 'positive', 'colorado'), ('FOOD#QUALITY', 'positive', 'fajitas'), ('FOOD#QUALITY', 'positive', 'NULL'), ('RESTAURANT#GENERAL', 'positive', 'place')])
("Everything I've had here is good, taco salads, burritos, enchiladas i love this place.", [('FOOD#QUALITY', 'positive', 'taco salads'), ('FOOD#QUALITY', 'positive', 'burritos'), ('FOOD#QUALITY', 'positive', 'enchiladas'), ('RESTAURANT#GENERAL', 'positive', 'place'), ('FOOD#QUALITY', 'positive', 'NULL')])
('– Great drinks , corn beef hash, coffee, B Fast burritos, Gluten Free menu.', [('DRINKS#QUALITY', 'positive', 'drinks'), ('FOOD#QUALITY', 'positive', 'corn beef hash'), ('DRINKS#QUALITY', 'positive', 'coffee'), ('FOOD#QUALITY', 'positive', 'B Fast burritos'), ('FOOD#STYLE_OPTIONS', 'positive', 'menu')])
('In the end our check cam

In [24]:
n = 0
for review in trail_data:
    for key, sentence_data in review.items():
        if n < len(sentence_data[1]):
            n = len(sentence_data[1])
n

4

In [10]:
def get_data_as_gpt_prompt(data):
    all_propmt_data = []
    for review in data:
        for key, sentence_data in review.items():
            prompt_data = {}
            prompt_data["text"] = sentence_data[0]
            opinions = []
            for opinion in sentence_data[1]:
                # print(opinion)/
                opinions.append({"target":opinion[2],"category": opinion[0], "polarity": opinion[1]})
            prompt_data["opinions"] = opinions
            all_propmt_data.append(prompt_data)
    return all_propmt_data
     

In [11]:
train_prompt_data = get_data_as_gpt_prompt(train_data)
test_prompt_data = get_data_as_gpt_prompt(test_data)
trail_prompt_data = get_data_as_gpt_prompt(trail_data)

In [12]:
print(len(train_prompt_data), len(test_prompt_data), len(trail_prompt_data))

1708 676 41


# Split into clasues

In [13]:
# !pip install openai

In [14]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="xxx",
)
     

In [15]:
import openai

def get_clauses_from_gpt(review_text, opinions):
    # Construct the input text with the review and opinions
    # opinions_text = "\n".join([f'Category: {op["category"]}, Polarity: {op["polarity"]}' for op in opinions])
    opinions_text = "\n".join([f'Target: {op["target"]}, Category: {op["category"]}, Polarity: {op["polarity"]}' for op in opinions])
    prompt = f"Split the following review into clauses and associate each clause with the given \
    (category, polarity) pairs. Must be a clause for each pair, no more, and the clasue also must contain \
    the target if possible. Keep the output format! :\n\nReview: {review_text}\n\nOpinions:\
    \n{opinions_text}\n\nOutput format:[(1, \"<clause_text>\", \"<category>\", \"<polarity>\")]"

    # Call the OpenAI API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an assistant that helps to process text by splitting\
              sentences into clauses and associating them with categories and polarities. The number of\
              caluses must match the number of pairs category-polarity. More must emerge from the clasue the\
              category and polarity, so it must have a verb at least."},
            {"role": "user", "content": prompt}
        ]
    )

    # return response['choices'][0]['message']['content'].strip()
    return response


In [ ]:
import ast

def proccess_gpt_result(result):
    message_content = result.choices[0].message.content
    message_content = message_content.replace('\n', '')
    start_tuple = message_content.find('(')
    end_tuple = message_content.find(')')
    next_t = message_content[start_tuple+1:].find('(')
    if next_t != -1 and next_t + start_tuple + 1 < end_tuple:
        end_tuple = message_content[end_tuple+1:].find(')') + end_tuple + 1
        
    data = []
    while start_tuple != -1 and end_tuple != -1 and start_tuple < end_tuple:
        data.append(ast.literal_eval(message_content[start_tuple: end_tuple+1]))
        # print(data)
        message_content = message_content[end_tuple+1:]
        start_tuple = message_content.find('(')
        end_tuple = message_content.find(')')
        next_t = message_content[start_tuple+1:].find('(')
        if next_t != -1 and next_t + start_tuple + 1 < end_tuple:
            # print("->",message_content, next_t + start_tuple, message_content[next_t + start_tuple + 1])
            # print("->",end_tuple, message_content[end_tuple], message_content[:end_tuple+1])
            end_tuple = message_content[end_tuple+1:].find(')') + end_tuple + 1
            # print("->",end_tuple, message_content[end_tuple], message_content[:end_tuple+1])
    return data

In [17]:
train_prompt_data[401]

{'text': "The place is small and intimate and you may feel a little crowded, but the service is excellent and it's great for friends out, a romantic date, or a special occassion.",
 'opinions': [{'target': 'service',
   'category': 'SERVICE#GENERAL',
   'polarity': 'positive'},
  {'target': 'place', 'category': 'AMBIENCE#GENERAL', 'polarity': 'negative'},
  {'target': 'place',
   'category': 'RESTAURANT#MISCELLANEOUS',
   'polarity': 'positive'}]}

In [19]:
result = get_clauses_from_gpt(train_prompt_data[184]["text"], train_prompt_data[184]["opinions"])

In [24]:
result.choices[0].message.content

'[(1, "service is rude", "SERVICE#GENERAL", "negative"), (2, "food is mediocre", "FOOD#QUALITY", "neutral"), (3, "there are tons of restaurants in NY...stay away from this one", "RESTAURANT#GENERAL", "negative")]'

In [434]:
res = proccess_gpt_result(result)
res

[(1, 'Rude service', 'SERVICE#GENERAL', 'negative'),
 (2, 'Medicore food', 'FOOD#QUALITY', 'neutral'),
 (3, 'Stay away from this one', 'RESTAURANT#GENERAL', 'negative')]

In [435]:
import pickle

In [436]:
# train data
X_train = []
count = 0
for sentence_data in train_prompt_data:
    if len(sentence_data["opinions"]) == 1:
        X_train.append((sentence_data["text"], sentence_data["opinions"][0]["category"], sentence_data["opinions"][0]["polarity"]))
    # result = get_clauses_from_gpt(sentence_data["text"], sentence_data["opinions"])
    if len(sentence_data["opinions"]) > 1:
        result = get_clauses_from_gpt(sentence_data["text"], sentence_data["opinions"])
        X_train += proccess_gpt_result(result)
    print(f"Done {count}")
    count+=1


with open("X_train_clauses", "wb") as fp:   #Pickling
    pickle.dump(X_train, fp)

Done 0
Done 1
Done 2
Done 3
Done 4
Done 5
Done 6
Done 7
Done 8
Done 9
Done 10
Done 11
Done 12
Done 13
Done 14
Done 15
Done 16
Done 17
Done 18
Done 19
Done 20
Done 21
Done 22
Done 23
Done 24
Done 25
Done 26
Done 27
Done 28
Done 29
Done 30
Done 31
Done 32
Done 33
Done 34
Done 35
Done 36
Done 37
Done 38
Done 39
Done 40
Done 41
Done 42
Done 43
Done 44
Done 45
Done 46
Done 47
Done 48
Done 49
Done 50
Done 51
Done 52
Done 53
Done 54
Done 55
Done 56
Done 57
Done 58
Done 59
Done 60
Done 61
Done 62
Done 63
Done 64
Done 65
Done 66
Done 67
Done 68
Done 69
Done 70
Done 71
Done 72
Done 73
Done 74
Done 75
Done 76
Done 77
Done 78
Done 79
Done 80
Done 81
Done 82
Done 83
Done 84
Done 85
Done 86
Done 87
Done 88
Done 89
Done 90
Done 91
Done 92
Done 93
Done 94
Done 95
Done 96
Done 97
Done 98
Done 99
Done 100
Done 101
Done 102
Done 103
Done 104
Done 105
Done 106
Done 107
Done 108
Done 109
Done 110
Done 111
Done 112
Done 113
Done 114
Done 115
Done 116
Done 117
Done 118
Done 119
Done 120
Done 121
Done 122
Don

In [437]:
X_train

[('Judging from previous posts this used to be a good place, but not any longer.',
  'RESTAURANT#GENERAL',
  'negative'),
 ('We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.',
  'SERVICE#GENERAL',
  'negative'),
 ('They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.',
  'SERVICE#GENERAL',
  'negative'),
 (1, 'The food was lousy', 'FOOD#QUALITY', 'negative'),
 (2, 'the portions tiny', 'FOOD#STYLE_OPTIONS', 'negative'),
 ('After all that, they complained to me about the small tip.',
  'SERVICE#GENERAL',
  'negative'),
 ('Avoid this place!', 'RESTAURANT#GENERAL', 'negative'),
 ('I have eaten at Saul, many times, the food is always consistently, outrageously good.',
  'FOOD#QUALITY',
  'positive'),
 ('Saul is the best restaurant on Smith Street and in Brooklyn.',
  'RESTAURANT#GENERAL',
  'positive'),
 (1, 'The duck confit is always a

In [438]:
# test data
X_test = []
count = 0
for sentence_data in test_prompt_data:
    if len(sentence_data["opinions"]) == 1:
        X_test.append((sentence_data["text"], sentence_data["opinions"][0]["category"], sentence_data["opinions"][0]["polarity"]))
    # result = get_clauses_from_gpt(sentence_data["text"], sentence_data["opinions"])
    if len(sentence_data["opinions"]) > 1:
        result = get_clauses_from_gpt(sentence_data["text"], sentence_data["opinions"])
        X_test += proccess_gpt_result(result)
    print(f"Done {count}")
    count+=1


with open("X_test_clauses", "wb") as fp:   #Pickling
    pickle.dump(X_test, fp)

Done 0
Done 1
Done 2
Done 3
Done 4
Done 5
Done 6
Done 7
Done 8
Done 9
Done 10
Done 11
Done 12
Done 13
Done 14
Done 15
Done 16
Done 17
Done 18
Done 19
Done 20
Done 21
Done 22
Done 23
Done 24
Done 25
Done 26
Done 27
Done 28
Done 29
Done 30
Done 31
Done 32
Done 33
Done 34
Done 35
Done 36
Done 37
Done 38
Done 39
Done 40
Done 41
Done 42
Done 43
Done 44
Done 45
Done 46
Done 47
Done 48
Done 49
Done 50
Done 51
Done 52
Done 53
Done 54
Done 55
Done 56
Done 57
Done 58
Done 59
Done 60
Done 61
Done 62
Done 63
Done 64
Done 65
Done 66
Done 67
Done 68
Done 69
Done 70
Done 71
Done 72
Done 73
Done 74
Done 75
Done 76
Done 77
Done 78
Done 79
Done 80
Done 81
Done 82
Done 83
Done 84
Done 85
Done 86
Done 87
Done 88
Done 89
Done 90
Done 91
Done 92
Done 93
Done 94
Done 95
Done 96
Done 97
Done 98
Done 99
Done 100
Done 101
Done 102
Done 103
Done 104
Done 105
Done 106
Done 107
Done 108
Done 109
Done 110
Done 111
Done 112
Done 113
Done 114
Done 115
Done 116
Done 117
Done 118
Done 119
Done 120
Done 121
Done 122
Don

In [439]:
# test data
X_trail = []
count = 0
for sentence_data in trail_prompt_data:
    if len(sentence_data["opinions"]) == 1:
        X_trail.append((sentence_data["text"], sentence_data["opinions"][0]["category"], sentence_data["opinions"][0]["polarity"]))
    # result = get_clauses_from_gpt(sentence_data["text"], sentence_data["opinions"])
    if len(sentence_data["opinions"]) > 1:
        result = get_clauses_from_gpt(sentence_data["text"], sentence_data["opinions"])
        X_trail += proccess_gpt_result(result)
    print(f"Done {count}")
    count+=1


with open("X_trail_clauses", "wb") as fp:   #Pickling
    pickle.dump(X_trail, fp)

Done 0
Done 1
Done 2
Done 3
Done 4
Done 5
Done 6
Done 7
Done 8
Done 9
Done 10
Done 11
Done 12
Done 13
Done 14
Done 15
Done 16
Done 17
Done 18
Done 19
Done 20
Done 21
Done 22
Done 23
Done 24
Done 25
Done 26
Done 27
Done 28
Done 29
Done 30
Done 31
Done 32
Done 33
Done 34
Done 35
Done 36
Done 37
Done 38
Done 39
Done 40
